In [12]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## request the page

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = requests.Session()
html = request.get(url)

soup = BeautifulSoup(html.content)

# the first table is the one!
table = soup.find_all('table')[0]

## check all cells and insert into a list

In [14]:
cells = []
for td in table.find_all('td'):
    cell = {}
    if td.span.text != 'Not assigned':
        cell['PostalCode'] = td.p.text[:3]
        cell['Borough'] = (td.span.text).split('(')[0]
        cell['Neighborhood'] = (((((td.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cells.append(cell)


## Add to dataframe

In [15]:

df=pd.DataFrame(cells)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Use geocoder to get coordinates

In [22]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
lat = []
lon = []

for i, postal_code in enumerate(df['PostalCode']):
    print('Coordinates for {}'.format(postal_code, end=''))

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print('{}{}'.format(i,lat_lng_coords), end='.')

        if lat_lng_coords:
            print('')
            lat.append(lat_lng_coords[0])
            lon.append(lat_lng_coords[1])

df['latitude'] = lat
df['longitude'] = lon

Coordinates for M3A
M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.M3A.

In [16]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [9]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Mountain View, CA'}
r = requests.get(url, params=params)
print(r.text)
#location = results[0]['geometry']['location']
#location['lat'], location['lng']

{
   "error_message" : "You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account",
   "results" : [],
   "status" : "REQUEST_DENIED"
}



In [17]:
import pandas as pd
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df2.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df3 = pd.merge(df, df2, on = "PostalCode", how = "inner")

In [19]:
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [23]:
print(df[df['PostalCode']=='M3A'])
print(df2[df2['PostalCode']=='M3A'])
print(df3[df3['PostalCode']=='M3A'])


  PostalCode     Borough Neighborhood
0        M3A  North York    Parkwoods
   PostalCode   Latitude  Longitude
25        M3A  43.753259 -79.329656
  PostalCode     Borough Neighborhood   Latitude  Longitude
0        M3A  North York    Parkwoods  43.753259 -79.329656
